In [2]:
import numpy as np
from dipy.io.streamline import load_trk
from dipy.viz import window, actor

In [3]:
def show_streamlines(streamlines): #function to visualize streamlines
    
    ren = window.Renderer()
    ren.add(actor.line(streamlines))
    window.show(ren)

In [4]:
def show_multiple_streamlines(streamline1,streamline2): #function to visualize streamlines
    
    ren = window.Renderer()
    ren.add(actor.line(streamline1))
    ren.add(actor.line(streamline2,colors=(0,1,1)))
    window.show(ren)

 load brain (tractogram)

In [5]:
brain, b_hr = load_trk("brain.trk")

load Arcuate Fasciculus left fiber bundle (AF_L)

In [6]:
bundle, bn_hr = load_trk("AF_L.trk")

In [7]:
#cst, bn_hr = load_trk("CST_recognized.trk")

let's visualize them

In [8]:
# first brain

#show_streamlines(brain)

In [9]:
# now bundle

#show_streamlines(bundle)

In [10]:
#show_streamlines(cst)

In [11]:
#show_multiple_streamlines(bundle,cst)

let's see its data type

In [12]:
#bundle # whole bundle, made of several streamlines

In [13]:
((brain)[0]).shape# number of streamlines in bundle

(331, 3)

In [14]:
# bundle[1] #one streamline in bundle
(np.array(bundle)[0]).shape
# np.array(bundle)

(331, 3)

In [15]:
(bundle[1].shape) #one point on one streamline

(557, 3)

In [16]:
npbrain = np.array(brain)

In [17]:
npbrain[0].shape

(331, 3)

In [18]:
len(bundle)

2000

these are x, y and z co-ordinates of the point

In [19]:
# set labels
labels = np.zeros(len(brain))
for i in range(len(bundle)):
    for j in range(len(brain)):
        if((npbrain[j].shape[0]==bundle[i].shape[0])):
#         print(i,j)
            if np.allclose(npbrain[j], bundle[i]):
                labels[j] = 1
                break
    if(i%100==0):
        print(i,end=" ")
                

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 

In [20]:
brain[0][1]

array([-53.05580139,  16.41918182,  44.91602325], dtype=float32)

In [21]:
bundle[0][1]

array([-53.05580139,  16.41918182,  44.91602325], dtype=float32)

In [22]:
# force put bundle data into the brain data
# for i in range(len(bundle)):
#     npbrain[i] = bundle[i]

In [23]:
# set labels
# labels = np.zeros(len(brain))
# for i in range(len(bundle)):
#     for j in range(len(brain)):
#         if((npbrain[j].shape[0]==bundle[i].shape[0])):
# #         print(i,j)
#             if np.allclose(npbrain[j], bundle[i]):
#                 print(j)
#                 labels[j] = 1
#                 break
#     if(i%1==0):
#         print("Not: "+ str(i))

In [24]:
npbrain.shape

(144000,)

In [25]:
n_pts = [len(streamline) for streamline in bundle]

In [26]:
from dipy.tracking.streamline import (set_number_of_points, nbytes,select_random_set_of_streamlines)
newStreamlines = set_number_of_points(bundle,101)
n_pts_ds = [len(streamline) for streamline in newStreamlines]

In [27]:
newBrain = set_number_of_points(brain,101)

In [28]:
newBrain[0].shape

(101, 3)

In [29]:
len(npbrain)

144000

In [30]:
import tensorflow as tf
import keras

Using TensorFlow backend.


In [31]:
# ==========
#   MODEL
# ==========

# Parameters
learning_rate = 0.02
training_steps = 7000
batch_size = 128
display_step = 200

# Network Parameters
seq_max_len = 400 # Sequence max length
n_hidden = 64 # hidden layer num of features
n_classes = 2 # linear sequence or not

trainset = npbrain
testset = labels

# tf Graph input
x = tf.placeholder("float", [None, seq_max_len, 1])
y = tf.placeholder("float", [None, n_classes])
# A placeholder for indicating each sequence length
seqlen = tf.placeholder(tf.int32, [None])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [32]:
def dynamicRNN(x, seqlen, weights, biases):
    tf.reset_default_graph()
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, seq_max_len, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)

    # Get lstm cell output, providing 'sequence_length' will perform dynamic
    # calculation.
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32,
                                sequence_length=seqlen)

    # When performing dynamic calculation, we must retrieve the last
    # dynamically computed output, i.e., if a sequence length is 10, we need
    # to retrieve the 10th output.
    # However TensorFlow doesn't support advanced indexing yet, so we build
    # a custom op that for each sample in batch size, get its length and
    # get the corresponding relevant output.

    # 'outputs' is a list of output at every timestep, we pack them in a Tensor
    # and change back dimension to [batch_size, n_step, n_input]
    outputs = tf.stack(outputs)
    outputs = tf.transpose(outputs, [1, 0, 2])

    # Hack to build the indexing and retrieve the right output.
    batch_size = tf.shape(outputs)[0]
    # Start indices for each sample
    index = tf.range(0, batch_size) * seq_max_len + (seqlen - 1)
    # Indexing
    outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), index)

    # Linear activation, using outputs computed above
    return tf.matmul(outputs, weights['out']) + biases['out']

In [33]:

pred = dynamicRNN(x, seqlen, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


ValueError: Tensor("rnn/Const:0", shape=(1,), dtype=int32) must be from the same graph as Tensor("ExpandDims:0", shape=(1,), dtype=int32).